In [6]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from gensim.models.phrases import Phrases, Phraser

# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('reuters')
from nltk.corpus import reuters

[nltk_data] Downloading package punkt to /home/statick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to /home/statick/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [7]:
documents = [reuters.raw(fileid).lower() for fileid in reuters.fileids()]

# Ver el primer documento
print(documents[0])

asian exporters fear damage from u.s.-japan rift
  mounting trade friction between the
  u.s. and japan has raised fears among many of asia's exporting
  nations that the row could inflict far-reaching economic
  damage, businessmen and officials said.
      they told reuter correspondents in asian capitals a u.s.
  move against japan might boost protectionist sentiment in the
  u.s. and lead to curbs on american imports of their products.
      but some exporters said that while the conflict would hurt
  them in the long-run, in the short-term tokyo's loss might be
  their gain.
      the u.s. has said it will impose 300 mln dlrs of tariffs on
  imports of japanese electronics goods on april 17, in
  retaliation for japan's alleged failure to stick to a pact not
  to sell semiconductors on world markets at below cost.
      unofficial japanese estimates put the impact of the tariffs
  at 10 billion dlrs and spokesmen for major electronics firms
  said they would virtually halt exports

In [8]:
tokens = [word for doc in documents for word in word_tokenize(doc)]
print(tokens[:15])

['asian', 'exporters', 'fear', 'damage', 'from', 'u.s.-japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'u.s.', 'and', 'japan']


In [9]:
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens)
finder.apply_freq_filter(10)
bigramas = finder.nbest(bigram_measures.pmi, n=50)

print(bigramas)

[('het', 'comite'), ('lago', 'agrio'), ('dar', 'es'), ('es', 'salaam'), ('hoare', 'govett'), ('corpus', 'christi'), ('paz', 'estenssoro'), ('corazon', 'aquino'), ('ay', 'expd-e'), ('lear', 'siegler'), ('l.f.', 'rothschild'), ('ranks', 'hovis'), ('abu', 'dhabi'), ('poison', 'pill'), ('hajime', 'tamura'), ('kleinwort', 'benson'), ('ind', 'ttl-f'), ('rjr', 'nabisco'), ('gates', 'learjet'), ('pro', 'forma'), ('margaret', 'thatcher'), ('carter', 'hawley'), ('canary', 'islands'), ('bra', 'kanon'), ('lord', 'abbett'), ('mcdonnell', 'douglas'), ('puerto', 'rico'), ('phelps', 'dodge'), ("'n", 'pak'), ('sao', 'paulo'), ('brace', 'jovanovich'), ('karl', 'otto'), ('marlin', 'fitzwater'), ('pizza', 'inn'), ('dean', 'witter'), ('buenos', 'aires'), ('costa', 'rica'), ('del', 'este'), ('king', 'fahd'), ('arturo', 'hernandez'), ('hernandez', 'grisanti'), ('pl', '480'), ('punta', 'del'), ('el', 'nino'), ('optional', 'origin'), ('du', 'pont'), ('drexel', 'burnham'), ('denis', 'bra'), ('hisham', 'nazer'),

In [10]:
sentences = [word_tokenize(sent) for sent in sent_tokenize("\n".join(documents).lower())]
sentences = [sent for sent in sentences if len(sent) > 1]

collocations = Phrases(sentences=sentences, min_count=10, threshold=0.5, scoring='npmi')
to_collocations = Phraser(collocations)

sent = 'new york is in united states of america. south africa and south america are in different continents'
print(to_collocations[word_tokenize(sent)])

['new_york', 'is', 'in', 'united_states', 'of', 'america', '.', 'south_africa', 'and', 'south', 'america', 'are', 'in', 'different', 'continents']


In [11]:
# Crear el objeto BigramCollocationFinder
collocations = BigramCollocationFinder.from_words(tokens)

# Usar BigramAssocMeasures para obtener las puntuaciones de los bigramas
scored = collocations.score_ngrams(BigramAssocMeasures().pmi)

# Crear un DataFrame con los bigramas y sus puntuaciones
df_collocations = pd.DataFrame(scored, columns=["bigram", "score"])

# Eliminar duplicados y ordenar por puntuación
df_collocations = df_collocations.drop_duplicates().sort_values(by="score", ascending=False)

# Imprimir los primeros 50 bigramas
print(df_collocations.head(50))

                            bigram      score
88                (165.4-r, 184.1)  20.560248
42              (1,842.6, 1,727.5)  20.560248
77                  (14.03, 13.33)  20.560248
84                  (16.45, 14.98)  20.560248
20          (1,500/1,700, 13-20/4)  20.560248
24               (1,589,6000, qms)  20.560248
25              (1,591.4, 1,635.6)  20.560248
26                (1,614,079, d/p)  20.560248
27            (1,630.40, 1,575.78)  20.560248
28                  (1,631, 1,934)  20.560248
45            (1,919.18, 1,920.13)  20.560248
46             (1,920.13, trade-x)  20.560248
47                  (1,933, 1,908)  20.560248
32            (1,663.69, 1,663.70)  20.560248
33              (1,679.0, 1,692.7)  20.560248
117             (2,767.4, 2,787.8)  20.560248
118               (2,797,000, tvi)  20.560248
119                 (2,799, 2,112)  20.560248
104           (2,028.45, 1,919.18)  20.560248
105             (2,064.8, 2,087.5)  20.560248
69                  (13.33, 13.40)

In [12]:
to_collocations.save('bigram_model')